In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import datetime  
from datetime import date 
import calendar 

In [2]:
# Read in csv file
csv_file = "../Resources/Crash_Information__Last_5_Years_.csv"
crash_df = pd.read_csv(csv_file)
crash_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145592 entries, 0 to 145591
Data columns (total 25 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   X                              145592 non-null  float64
 1   Y                              145592 non-null  float64
 2   OBJECTID                       145592 non-null  int64  
 3   ACC_ID                         145592 non-null  int64  
 4   ROAD_NO                        145592 non-null  object 
 5   ROAD_NAME                      145592 non-null  object 
 6   COMMON_ROAD_NAME               145592 non-null  object 
 7   CWAY                           145592 non-null  object 
 8   SLK                            145592 non-null  float64
 9   INTERSECTION_NO                77094 non-null   float64
 10  INTERSECTION_DESC              77094 non-null   object 
 11  LONGITUDE                      145592 non-null  float64
 12  LATITUDE                      

In [3]:
# Drop irrelevant columns
cleaned_crash_df = crash_df.drop(["X","Y","OBJECTID","ROAD_NAME","SLK","INTERSECTION_NO","INTERSECTION_DESC"],inplace=False,axis=1)
cleaned_crash_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145592 entries, 0 to 145591
Data columns (total 18 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   ACC_ID                         145592 non-null  int64  
 1   ROAD_NO                        145592 non-null  object 
 2   COMMON_ROAD_NAME               145592 non-null  object 
 3   CWAY                           145592 non-null  object 
 4   LONGITUDE                      145592 non-null  float64
 5   LATITUDE                       145592 non-null  float64
 6   CRASH_DATE                     145592 non-null  object 
 7   CRASH_TIME                     145124 non-null  float64
 8   ACCIDENT_TYPE                  145592 non-null  object 
 9   SEVERITY                       145592 non-null  object 
 10  EVENT_NATURE                   137488 non-null  object 
 11  EVENT_TYPE                     37065 non-null   object 
 12  TOTAL_BIKE_INVOLVED           

In [4]:
cleaned_crash_df=cleaned_crash_df.dropna(subset=['CRASH_TIME'])
cleaned_crash_df['CRASH_TIME'].unique()

array([ 130., 1345., 1230., ...,  337.,  614.,  413.])

In [5]:
cleaned_crash_df['CRASH_TIME'].isnull().values.any()

False

In [6]:
cleaned_crash_df["CRASH_DATE"].str[-7:].str[0:2]

0         06
1         06
2         06
3         06
4         06
          ..
145587    07
145588    07
145589    07
145590    07
145591    07
Name: CRASH_DATE, Length: 145124, dtype: object

In [7]:
cleaned_crash_df["CRASH_DATE"].str[-7:].str[0:2].astype(str)

0         06
1         06
2         06
3         06
4         06
          ..
145587    07
145588    07
145589    07
145590    07
145591    07
Name: CRASH_DATE, Length: 145124, dtype: object

In [8]:
cleaned_crash_df["CRASH_TIME_HRS"] = cleaned_crash_df["CRASH_TIME"].div(100).apply(np.floor).astype(int)
cleaned_crash_df["CRASH_TIME_MIN"] = cleaned_crash_df["CRASH_TIME"].mod(100).astype(int)
cleaned_crash_df["YEAR"]=cleaned_crash_df["CRASH_DATE"].str[-4:].astype(int)
cleaned_crash_df["MONTH"]=cleaned_crash_df["CRASH_DATE"].str[-7:].str[0:2].astype(str)

cleaned_crash_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 145124 entries, 0 to 145591
Data columns (total 22 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   ACC_ID                         145124 non-null  int64  
 1   ROAD_NO                        145124 non-null  object 
 2   COMMON_ROAD_NAME               145124 non-null  object 
 3   CWAY                           145124 non-null  object 
 4   LONGITUDE                      145124 non-null  float64
 5   LATITUDE                       145124 non-null  float64
 6   CRASH_DATE                     145124 non-null  object 
 7   CRASH_TIME                     145124 non-null  float64
 8   ACCIDENT_TYPE                  145124 non-null  object 
 9   SEVERITY                       145124 non-null  object 
 10  EVENT_NATURE                   137065 non-null  object 
 11  EVENT_TYPE                     36903 non-null   object 
 12  TOTAL_BIKE_INVOLVED           

In [9]:
cleaned_crash_df1=cleaned_crash_df.replace(np.nan,'Others',regex=True)
cleaned_crash_df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 145124 entries, 0 to 145591
Data columns (total 22 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   ACC_ID                         145124 non-null  int64  
 1   ROAD_NO                        145124 non-null  object 
 2   COMMON_ROAD_NAME               145124 non-null  object 
 3   CWAY                           145124 non-null  object 
 4   LONGITUDE                      145124 non-null  float64
 5   LATITUDE                       145124 non-null  float64
 6   CRASH_DATE                     145124 non-null  object 
 7   CRASH_TIME                     145124 non-null  float64
 8   ACCIDENT_TYPE                  145124 non-null  object 
 9   SEVERITY                       145124 non-null  object 
 10  EVENT_NATURE                   145124 non-null  object 
 11  EVENT_TYPE                     145124 non-null  object 
 12  TOTAL_BIKE_INVOLVED           

In [12]:
# Add day of the week column
def findDay(date): 
    day, month, year = (int(i) for i in date.split('/'))     
    dayWeek = datetime.date(year, month, day) 
    return dayWeek.strftime("%A") 

# date = '07/10/2020'
# print(findDay(date)) 

cleaned_crash_df1['CRASH_DAYWEEK'] = cleaned_crash_df1['CRASH_DATE'].apply(findDay)

In [21]:
# Add time of day column
def findTimeDay(hour): 
    if hour >= 0 and hour <= 5:
        return("Night")
    elif hour >= 6 and hour < 12:
        return("Morning")
    elif hour >= 12 and hour < 18:
        return("Afternoon")
    elif hour >= 18 and hour < 21:
        return("Evening")
    else:
        return("Night")

# hour = 11
# print(findTimeDay(hour))

cleaned_crash_df1['CRASH_TIMEDAY'] = cleaned_crash_df1['CRASH_TIME_HRS'].apply(findTimeDay)

In [23]:
# Final check
cleaned_crash_df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 145124 entries, 0 to 145591
Data columns (total 24 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   ACC_ID                         145124 non-null  int64  
 1   ROAD_NO                        145124 non-null  object 
 2   COMMON_ROAD_NAME               145124 non-null  object 
 3   CWAY                           145124 non-null  object 
 4   LONGITUDE                      145124 non-null  float64
 5   LATITUDE                       145124 non-null  float64
 6   CRASH_DATE                     145124 non-null  object 
 7   CRASH_TIME                     145124 non-null  float64
 8   ACCIDENT_TYPE                  145124 non-null  object 
 9   SEVERITY                       145124 non-null  object 
 10  EVENT_NATURE                   145124 non-null  object 
 11  EVENT_TYPE                     145124 non-null  object 
 12  TOTAL_BIKE_INVOLVED           

In [27]:
cleaned_crash_df1.to_csv("../database/crash_data_final.csv")